# Lab) Kaggle Competition for Beginner

#### Data Preprocess
- Data Load
- Data Set of Train, Valid, Test
- Feature Engineerning (To Do)
- Data Scaling & Encoding

#### Model Tranding
- Hyper Parameter Tune: Optuna
- Model Traning

#### Ensemble
- Out-of-fold(OOF) Ensemble

#### Submission
- Make submission

In [1]:
# 나눔 글꼴 설치
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 6 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (7,281 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120882 files and direc

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [51]:
import os
from os.path import join
import multiprocessing

import numpy as np
import pandas as pd

# 시각화
import matplotlib.pylab as plt
import matplotlib as mpl
import seaborn as sns
import re

%matplotlib inline
#plt.rc('font', family='NanumBarunGothic')

import warnings

warnings.filterwarnings('ignore')

n_cpus = multiprocessing.cpu_count() - 1

# Data Preprocess


## 1. Data Load
- pandas.read_csv()

In [52]:
ROOT_PATH = '/content/drive/MyDrive/ASAC/kaggle특강/실습/'

In [53]:
train_path = join(ROOT_PATH, 'data', 'train.csv')
test_path  = join(ROOT_PATH, 'data', 'test.csv')
submission_path = join(ROOT_PATH, 'data', 'sample_submission.csv')

In [54]:
data = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [55]:
data.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8


In [56]:
test.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
0,74051,I,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552
1,74052,I,1.1625,0.8875,0.2750,15.507176,7.030676,3.246018,3.968930
2,74053,F,1.2875,0.9875,0.3250,14.571643,5.556502,3.883882,4.819415
3,74054,F,1.5500,0.9875,0.3875,28.377849,13.380964,6.548735,7.030676
4,74055,I,1.1125,0.8500,0.2625,11.765042,5.528153,2.466407,3.331066


데이터 설명
- id: 각 게 데이터의 고유한 식별자로, 데이터를 구분하는데 사용됩니다.
- Sex: 게의 성별을 나타내는 컬럼으로, 주로 두 가지 값으로 표현됩니다. 예를 들어, "M"은 수컷(male)을, "F"는 암컷(female)을 나타낼 수 있습니다.., "I"는 불확실한 개체
- Length: 게의 길이를 나타내는 컬럼으로, 일반적으로 밀리미터(mm) 단위로 기록됩니다.
- Diameter: 게의 지름을 나타내는 컬럼으로, 마찬가지로 밀리미터(mm) 단위로 기록됩니다.
- Height: 게의 높이를 나타내는 컬럼으로, 역시 밀리미터(mm) 단위로 기록됩니다.
- Weight: 게의 총 무게를 나타내는 컬럼으로, 일반적으로 그램(g) 단위로 표시됩니다.
- Shucked Weight: 게의 껍질을 제거한 후의 무게를 나타내는 컬럼으로, 그램(g) 단위로 기록됩니다.
- Viscera Weight: 게 내장의 무게를 나타내는 컬럼으로, 그램(g) 단위로 표현됩니다.
- Shell Weight: 게의 껍질의 무게를 나타내는 컬럼으로, 그램(g) 단위로 기록됩니다.
- Age(Target): 게의 나이를 나타내는 컬럼으로, 주로 "년"이나 "년수"로 표시되며, 게의 연령을 추정하기 위해 사용됩니다.

In [57]:
# 라벨 컬럼을 별도의 변수로 빼둡니다.
label = data['Age']

In [58]:
data.head(20)

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8
5,5,M,1.5000,1.1750,0.4125,28.845616,13.409313,6.789705,7.937860,10
6,6,M,1.5750,1.1375,0.3500,30.022120,11.935140,7.342521,8.646598,11
7,7,I,1.3125,1.0250,0.3500,18.299602,8.249704,3.898056,5.669900,11
8,8,F,1.6000,1.2875,0.4375,38.824640,16.967176,7.413394,10.772810,12
9,9,M,1.0250,0.7625,0.2625,10.305043,4.493396,2.126212,2.976698,11


In [59]:
data.describe()#include='all')

,id,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
count,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000
mean,37025.000000,1.317460,1.024496,0.348089,23.385217,10.104270,5.058386,6.723870,9.967806
std,21376.826729,0.287757,0.237396,0.092034,12.648153,5.618025,2.792729,3.584372,3.175189
min,0.000000,0.187500,0.137500,0.000000,0.056699,0.028349,0.042524,0.042524,1.000000
25%,18512.500000,1.150000,0.887500,0.300000,13.437663,5.712424,2.863300,3.968930,8.000000
50%,37025.000000,1.375000,1.075000,0.362500,23.799405,9.908150,4.989512,6.931453,10.000000
75%,55537.500000,1.537500,1.200000,0.412500,32.162508,14.033003,6.988152,9.071840,11.000000
max,74050.000000,2.012815,1.612500,2.825000,80.101512,42.184056,21.545620,28.491248,29.000000


In [60]:
#
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74051 entries, 0 to 74050
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              74051 non-null  int64  
 1   Sex             74051 non-null  object 
 2   Length          74051 non-null  float64
 3   Diameter        74051 non-null  float64
 4   Height          74051 non-null  float64
 5   Weight          74051 non-null  float64
 6   Shucked Weight  74051 non-null  float64
 7   Viscera Weight  74051 non-null  float64
 8   Shell Weight    74051 non-null  float64
 9   Age             74051 non-null  int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 5.6+ MB


In [61]:
test.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
0,74051,I,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552
1,74052,I,1.1625,0.8875,0.2750,15.507176,7.030676,3.246018,3.968930
2,74053,F,1.2875,0.9875,0.3250,14.571643,5.556502,3.883882,4.819415
3,74054,F,1.5500,0.9875,0.3875,28.377849,13.380964,6.548735,7.030676
4,74055,I,1.1125,0.8500,0.2625,11.765042,5.528153,2.466407,3.331066


In [62]:
test.describe()

,id,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
count,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000,49368.000000
mean,98734.500000,1.319795,1.026357,0.348693,23.464627,10.130220,5.072615,6.745876
std,14251.458382,0.286950,0.236593,0.092259,12.641703,5.611779,2.791563,3.575650
min,74051.000000,0.187500,0.137500,0.000000,0.056699,0.028349,0.014175,0.042524
25%,86392.750000,1.162500,0.887500,0.300000,13.536886,5.740774,2.877474,3.968930
50%,98734.500000,1.387500,1.075000,0.362500,23.813580,9.979024,4.989512,6.945627
75%,111076.250000,1.537500,1.200000,0.412500,32.290081,14.047177,7.016501,9.171063
max,123418.000000,2.037500,1.625000,2.825000,80.101512,42.184056,21.545620,28.491248


In [63]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49368 entries, 0 to 49367
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              49368 non-null  int64  
 1   Sex             49368 non-null  object 
 2   Length          49368 non-null  float64
 3   Diameter        49368 non-null  float64
 4   Height          49368 non-null  float64
 5   Weight          49368 non-null  float64
 6   Shucked Weight  49368 non-null  float64
 7   Viscera Weight  49368 non-null  float64
 8   Shell Weight    49368 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 3.4+ MB


## Feature Engineering

### 불필요한 컬럼 삭제
- id 컬럼은 식별자의 역할을 할 뿐 유의미한 feature로 만들기 어렵습니다. 제거하도록 합니다.

In [64]:
data.drop(columns=['id', 'Age'], inplace=True)

test.drop(columns=['id'], inplace=True)

### 1)  성별에 따른 평균과의 편차 피쳐 추가

- 성별 특징을 강조하기 위해 각 성별 평균에서 각 데이터의 값의 차이를 구한다.

In [65]:
tmp_train_sex = data.groupby(['Sex']).mean().reset_index()
tmp_test_sex = test.groupby(['Sex']).mean().reset_index()

In [66]:
tmp_train_sex.columns  = 'mean_diff_' + tmp_train_sex.columns
tmp_train_sex = tmp_train_sex.rename(columns = {'mean_diff_Sex': 'Sex'})

In [67]:
tmp_train = data.merge(tmp_train_sex, how = 'left', on = 'Sex')

In [68]:
tmp_train['mean_diff_Length'] = tmp_train['mean_diff_Length'] - tmp_train['Length']
tmp_train['mean_diff_Diameter'] = tmp_train['mean_diff_Diameter'] - tmp_train['Diameter']
tmp_train['mean_diff_Height'] = tmp_train['mean_diff_Height'] - tmp_train['Height']
tmp_train['mean_diff_Weight'] = tmp_train['mean_diff_Weight'] - tmp_train['Weight']
tmp_train['mean_diff_Shucked Weight'] = tmp_train['mean_diff_Shucked Weight'] - tmp_train['Shucked Weight']
tmp_train['mean_diff_Viscera Weight'] = tmp_train['mean_diff_Viscera Weight'] - tmp_train['Viscera Weight']
tmp_train['mean_diff_Shell Weight'] = tmp_train['mean_diff_Shell Weight'] - tmp_train['Shell Weight']

In [69]:
tmp_test_sex.columns  = 'mean_diff_' + tmp_test_sex.columns
tmp_test_sex = tmp_test_sex.rename(columns = {'mean_diff_Sex': 'Sex'})
tmp_test = test.merge(tmp_test_sex, how = 'left', on = 'Sex')
tmp_test['mean_diff_Length'] = tmp_test['mean_diff_Length'] - tmp_test['Length']
tmp_test['mean_diff_Diameter'] = tmp_test['mean_diff_Diameter'] - tmp_test['Diameter']
tmp_test['mean_diff_Height'] = tmp_test['mean_diff_Height'] - tmp_test['Height']
tmp_test['mean_diff_Weight'] = tmp_test['mean_diff_Weight'] - tmp_test['Weight']
tmp_test['mean_diff_Shucked Weight'] = tmp_test['mean_diff_Shucked Weight'] - tmp_test['Shucked Weight']
tmp_test['mean_diff_Viscera Weight'] = tmp_test['mean_diff_Viscera Weight'] - tmp_test['Viscera Weight']
tmp_test['mean_diff_Shell Weight'] = tmp_test['mean_diff_Shell Weight'] - tmp_test['Shell Weight']

In [70]:
tmp_train.columns == tmp_test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [71]:
tmp_train.shape, tmp_test.shape

((74051, 15), (49368, 15))

### 2) Crab의 덩치 Length * Daimeter (길이 * 지름)
- https://www.kaggle.com/code/arunklenin/ps3e16-eda-feature-engineering-ensemble 참고 및 변형

In [72]:
tmp_train["crab_area"] = tmp_train["Length"]*tmp_train["Diameter"]
tmp_test["crab_area"] = tmp_test["Length"]*tmp_test["Diameter"]

### 3) Crab의 밀도 (무게 / 덩치 * 키)
- https://www.kaggle.com/code/arunklenin/ps3e16-eda-feature-engineering-ensemble 참고 및 변형

In [73]:
tmp_train["density"] = tmp_train['Weight']/(tmp_train['crab_area']*tmp_train['Height'])
tmp_test["density"] = tmp_test['Weight']/(tmp_test['crab_area']*tmp_test['Height'])

### 4) Crab의 BMI (무게 / 키**2)
- https://www.kaggle.com/code/arunklenin/ps3e16-eda-feature-engineering-ensemble 참고 및 변형

In [74]:
tmp_train['bmi']=tmp_train['Weight']/(tmp_train['Height']**2)
tmp_test['bmi']=tmp_test['Weight']/(tmp_test['Height']**2)

###5) 무게 대비 껍질 제거 무게 (Shucked Weight / Weight)

In [75]:
tmp_train["weight_no_shucked"] = tmp_train["Shucked Weight"] / tmp_train["Weight"]
tmp_test["weight_no_shucked"] = tmp_test["Shucked Weight"] / tmp_test["Weight"]

###6) 무게 대비 내장 무게 (Viscera Weight / Weight)

In [76]:
tmp_train["weight_no_Viscera"] = tmp_train["Viscera Weight"] / tmp_train["Weight"]
tmp_test["weight_no_Viscera"] = tmp_test["Viscera Weight"] / tmp_test["Weight"]

### 7) 무게 대비 껍질 무게 (Shell Weight / Weight)

In [77]:
tmp_train["weight_no_Shell"] = tmp_train["Shell Weight"] / tmp_train["Weight"]
tmp_test["weight_no_Shell"] = tmp_test["Shell Weight"] / tmp_test["Weight"]

In [78]:
tmp_train.columns == tmp_test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [79]:
tmp_train.shape, tmp_test.shape

((74051, 21), (49368, 21))

### 8) 순수 무게 (Weight - (Viscera Weight + Shell Weight))

- 0보다 작을 수 있나?
- 음수인 값들은 0 혹은 최솟값으로 대체하는것이?

In [80]:
tmp_train["weight_no"] = tmp_train["Weight"] - (tmp_train["Shell Weight"] + tmp_train["Viscera Weight"])
tmp_test["weight_no"] = tmp_test["Weight"] - (tmp_test["Shell Weight"] + tmp_test["Viscera Weight"])

In [81]:
sum(tmp_train["weight_no"] < 0)

22

In [82]:
tmp_train['weight_no'] = tmp_train.apply(lambda row: row['weight_no'] if row['weight_no'] >= 0 else tmp_train.groupby('Sex')['weight_no'].transform('mean'), axis=1)
tmp_test['weight_no'] = tmp_test.apply(lambda row: row['weight_no'] if row['weight_no'] >= 0 else tmp_test.groupby('Sex')['weight_no'].transform('mean'), axis=1)

In [83]:
tmp_train['weight_no'] = pd.to_numeric(tmp_train['weight_no'], errors='coerce')
tmp_test['weight_no'] = pd.to_numeric(tmp_test['weight_no'], errors='coerce')

In [84]:
sum(tmp_train.weight_no < 0)

0

In [85]:
sum(tmp_test.weight_no < 0)

0

## 전처리

### 수치형 변수 컬럼 및 범주형 변수 컬럼

In [86]:
# DF.info에서 Object type을 가진 컬럼은 모두 범주형 변수
cat_columns = data.select_dtypes(include='object').columns
num_columns = data.select_dtypes(exclude='object').columns

print('Categorical Columns: \n{}\n\n Numeric Columns: \n{}\n'.format(cat_columns, num_columns))

Categorical Columns: 
Index(['Sex'], dtype='object')

 Numeric Columns: 
Index(['Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight',
       'Viscera Weight', 'Shell Weight'],
      dtype='object')



In [87]:
tmp_train

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,mean_diff_Length,mean_diff_Diameter,...,mean_diff_Shucked Weight,mean_diff_Viscera Weight,mean_diff_Shell Weight,crab_area,density,bmi,weight_no_shucked,weight_no_Viscera,weight_no_Shell,weight_no
0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,-0.478634,-0.376158,...,-7.592119,-4.158520,-4.931453,1.791875,43.117872,206.031566,0.439335,0.229452,0.288160,13.976304
1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,-0.053634,-0.026158,...,0.615061,0.164779,0.015535,0.907500,41.746822,137.764512,0.434014,0.223129,0.326531,4.691842
2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,0.044193,0.006754,...,0.876709,0.552248,1.405104,1.543594,42.804808,176.195292,0.457666,0.224256,0.268879,12.558829
3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,-0.234747,-0.264598,...,-7.564985,-4.495112,-6.411757,2.401250,42.195154,202.642226,0.401791,0.216970,0.296027,24.671833
4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,-0.203634,-0.213658,...,-6.840857,-2.018132,-2.535920,1.265625,54.522298,204.458617,0.514303,0.193548,0.255630,12.828149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74046,F,1.6625,1.2625,0.4375,50.660556,20.680960,10.361742,12.332033,-0.197247,-0.114598,...,-7.891005,-3.865015,-3.746904,2.098906,55.169476,264.675560,0.408226,0.204533,0.243425,27.966782
74047,I,1.0750,0.8625,0.2750,10.446791,4.323299,2.296310,3.543687,-0.028634,-0.063658,...,0.813508,0.193129,-0.126212,0.927188,40.971573,138.139382,0.413840,0.219810,0.339213,4.606794
74048,F,1.4875,1.2000,0.4125,29.483480,12.303683,7.540967,8.079607,-0.022247,-0.052098,...,0.486273,-1.044240,0.505521,1.785000,40.042075,173.272977,0.417308,0.255769,0.274038,13.862906
74049,I,1.2125,0.9625,0.3125,16.768729,8.972617,2.919999,4.280774,-0.166134,-0.163658,...,-3.835810,-0.430560,-0.863299,1.167031,45.979860,171.711788,0.535080,0.174134,0.255283,9.567956


In [88]:
tmp_test

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,mean_diff_Length,mean_diff_Diameter,...,mean_diff_Shucked Weight,mean_diff_Viscera Weight,mean_diff_Shell Weight,crab_area,density,bmi,weight_no_shucked,weight_no_Viscera,weight_no_Shell,weight_no
0,I,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552,-0.000009,0.039597,...,1.520842,0.778643,0.727369,0.800625,39.143274,113.960304,0.424342,0.200658,0.315789,4.167376
1,I,1.1625,0.8875,0.2750,15.507176,7.030676,3.246018,3.968930,-0.112509,-0.085403,...,-1.852748,-0.738055,-0.520009,1.031719,54.656109,205.053574,0.453382,0.209324,0.255941,8.292229
2,F,1.2875,0.9875,0.3250,14.571643,5.556502,3.883882,4.819415,0.179316,0.161324,...,7.250517,2.623446,3.771875,1.271406,35.264751,137.956383,0.381323,0.266537,0.330739,5.868346
3,F,1.5500,0.9875,0.3875,28.377849,13.380964,6.548735,7.030676,-0.083184,0.161324,...,-0.573945,-0.041407,1.560614,1.530625,47.845266,188.988800,0.471528,0.230769,0.247752,14.798439
4,I,1.1125,0.8500,0.2625,11.765042,5.528153,2.466407,3.331066,-0.062509,-0.047903,...,-0.350225,0.041556,0.117854,0.945625,47.396388,170.739846,0.469880,0.209639,0.283133,5.967570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49363,F,1.3000,1.0375,0.3250,16.315137,6.690482,5.173784,3.756309,0.166816,0.111324,...,6.116537,1.333544,4.834981,1.348750,37.219961,154.462838,0.410078,0.317116,0.230235,7.385045
49364,I,1.0375,0.7625,0.2625,10.276694,4.436697,1.998640,3.543687,0.012491,0.039597,...,0.741231,0.509323,-0.094767,0.791094,49.487573,149.140227,0.431724,0.194483,0.344828,4.734367
49365,F,1.4875,1.1625,0.3625,31.382897,11.396499,6.846404,8.788345,-0.020684,-0.013676,...,1.410520,-0.339076,-0.197055,1.729219,50.065099,238.823469,0.363144,0.218157,0.280036,15.748147
49366,F,1.2375,0.9500,0.2875,15.663099,6.095142,3.727959,4.961163,0.229316,0.198824,...,6.711877,2.779369,3.630127,1.175625,46.341600,189.496847,0.389140,0.238009,0.316742,6.973977


# 모델링 (Pycaret)

In [40]:
!pip install -q pycaret
!pip install --upgrade -q xgboost
!pip install -q catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.7/484.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 126.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 84.3 MB/s

In [42]:
!pip install scipy

In [89]:
from pycaret.regression import *

X = pd.concat([tmp_train, label], axis = 1)

In [90]:
X.head()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,mean_diff_Length,mean_diff_Diameter,...,mean_diff_Viscera Weight,mean_diff_Shell Weight,crab_area,density,bmi,weight_no_shucked,weight_no_Viscera,weight_no_Shell,weight_no,Age
0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,-0.478634,-0.376158,...,-4.158520,-4.931453,1.791875,43.117872,206.031566,0.439335,0.229452,0.288160,13.976304,9
1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,-0.053634,-0.026158,...,0.164779,0.015535,0.907500,41.746822,137.764512,0.434014,0.223129,0.326531,4.691842,8
2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,0.044193,0.006754,...,0.552248,1.405104,1.543594,42.804808,176.195292,0.457666,0.224256,0.268879,12.558829,9
3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,-0.234747,-0.264598,...,-4.495112,-6.411757,2.401250,42.195154,202.642226,0.401791,0.216970,0.296027,24.671833,11
4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,-0.203634,-0.213658,...,-2.018132,-2.535920,1.265625,54.522298,204.458617,0.514303,0.193548,0.255630,12.828149,8


In [91]:
for col in X.columns[1:]:  # 2번째 칼럼부터 시작
    X[col] = pd.to_numeric(X[col], errors='coerce')  # 숫자로 변환 (변환 불가능한 값은 NaN으로 처리)

# 0보다 작은 값이면 해당 열의 평균값으로 대체
for col in X.columns[1:]:
    mask = X['weight_no'] < 0
    col_mean = X.loc[~mask, col].mean()  # 0보다 작지 않은 값들의 평균
    X.loc[mask, col] = col_mean  # 0보다 작은 값은 평균값으로 대체

In [100]:
X.isin([np.inf, -np.inf]).sum()

Sex                         0
Length                      0
Diameter                    0
Height                      0
Weight                      0
Shucked Weight              0
Viscera Weight              0
Shell Weight                0
mean_diff_Length            0
mean_diff_Diameter          0
mean_diff_Height            0
mean_diff_Weight            0
mean_diff_Shucked Weight    0
mean_diff_Viscera Weight    0
mean_diff_Shell Weight      0
crab_area                   0
density                     0
bmi                         0
weight_no_shucked           0
weight_no_Viscera           0
weight_no_Shell             0
weight_no                   0
Age                         0
dtype: int64

In [104]:
from matplotlib.colors import Normalize
reg = setup(data = X # 데이터
      ,target = 'Age' # Target
      ,verbose = True # 진행 로그 출력
      ,train_size = 0.7 # 내부적으로 검증시 훈련데이터의 비율
      ,data_split_shuffle = True # 데이터 split시 데이터를 섞을것인지 여부
      ,normalize = True # 정규화 여부
      ,session_id = 100 # 난수 시드
      )

,Description,Value
0,Session id,100
1,Target,Age
2,Target type,Regression
3,Original data shape,"(74051, 23)"
4,Transformed data shape,"(74051, 25)"
5,Transformed train set shape,"(51835, 25)"
6,Transformed test set shape,"(22216, 25)"
7,Numeric features,21
8,Categorical features,1
9,Rows with missing values,0.1%


In [ ]:
models()

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


In [ ]:
#top_5_model = compare_models(fold = 5, round = 3, sort = 'mse', n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,1.407,4.212,2.052,0.582,0.163,0.137,0.436
gbr,Gradient Boosting Regressor,1.423,4.260,2.064,0.577,0.164,0.139,5.158
catboost,CatBoost Regressor,1.422,4.265,2.065,0.577,0.164,0.139,9.524
xgboost,Extreme Gradient Boosting,1.431,4.361,2.088,0.567,0.165,0.139,0.728
rf,Random Forest Regressor,1.471,4.486,2.118,0.555,0.170,0.145,4.542
lr,Linear Regression,1.482,4.505,2.122,0.553,0.169,0.145,0.186
ridge,Ridge Regression,1.482,4.505,2.122,0.553,0.169,0.145,0.172
lar,Least Angle Regression,1.482,4.505,2.122,0.553,0.169,0.145,0.164
br,Bayesian Ridge,1.482,4.505,2.122,0.553,0.169,0.145,0.242
et,Extra Trees Regressor,1.482,4.525,2.127,0.551,0.171,0.146,2.112


Processing:   0%|          | 0/89 [00:00<?, ?it/s]

In [105]:
model_rf = create_model('rf', fold = 3)
model_cb = create_model('catboost', fold = 3)
model_lgb = create_model('lightgbm', fold = 3)
model_xgb = create_model('xgboost', fold = 3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.4735,4.4440,2.1081,0.5619,0.1690,0.1450
1,1.4592,4.4018,2.0980,0.5601,0.1680,0.1439
2,1.4658,4.4341,2.1057,0.5593,0.1687,0.1441
Mean,1.4662,4.4266,2.1040,0.5604,0.1685,0.1444
Std,0.0059,0.0180,0.0043,0.0011,0.0004,0.0005


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.4218,4.2356,2.0581,0.5824,0.1639,0.1387
1,1.4011,4.1803,2.0446,0.5823,0.1625,0.1369
2,1.4153,4.2490,2.0613,0.5777,0.1635,0.1374
Mean,1.4127,4.2216,2.0546,0.5808,0.1633,0.1377
Std,0.0087,0.0297,0.0072,0.0022,0.0006,0.0008


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.4146,4.2134,2.0527,0.5846,0.1634,0.1381
1,1.4034,4.1963,2.0485,0.5807,0.1625,0.1370
2,1.4106,4.2560,2.0630,0.5770,0.1634,0.1369
Mean,1.4095,4.2219,2.0547,0.5808,0.1631,0.1373
Std,0.0046,0.0251,0.0061,0.0031,0.0004,0.0005


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.4627,4.4656,2.1132,0.5597,0.1682,0.1426
1,1.4365,4.4068,2.0992,0.5596,0.1666,0.1401
2,1.4467,4.4751,2.1155,0.5553,0.1679,0.1405
Mean,1.4486,4.4492,2.1093,0.5582,0.1676,0.1411
Std,0.0108,0.0302,0.0072,0.0021,0.0007,0.0011


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# tune_model_cb = tune_model(model_cb, fold = 3, optimize = 'mae', choose_better = True)
# tune_model_lgb = tune_model(model_lgb, fold = 3, optimize = 'mae', choose_better = True)
# tune_model_xgb = tune_model(model_xgb, fold = 3, optimize = 'mae', choose_better = True)

In [106]:
candidate_models = [model_cb, model_lgb, model_xgb, model_rf]

In [108]:
stacker = stack_models(candidate_models, choose_better = False)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,06:51:24
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Stacking Regressor


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
# stacker_false = stack_models(candidate_models, fold = 10, choose_better = True, restack = False)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.4113,4.2196,2.0542,0.5844,0.1618,0.1364
1,1.3899,4.0662,2.0165,0.5898,0.1612,0.1367
2,1.4282,4.2533,2.0624,0.5852,0.1651,0.1399
3,1.4419,4.4827,2.1172,0.5641,0.1649,0.1377
4,1.3980,4.1692,2.0419,0.5906,0.1629,0.1377
5,1.3508,3.7444,1.9350,0.6159,0.1560,0.1340
6,1.3897,4.2037,2.0503,0.5766,0.1611,0.1338
7,1.4186,4.3272,2.0802,0.5873,0.1627,0.1360
8,1.4088,4.1966,2.0486,0.5774,0.1631,0.1387


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
stack_model = finalize_model(stacker)
stack_model

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Length', 'Diameter', 'Height',
                                             'Weight', 'Shucked Weight',
                                             'Viscera Weight', 'Shell Weight'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['Sex'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('onehot_encoding',
                 Transforme...
                                                             max_delta_step=None,
                                                             max_depth=None,
                                                             max_leaves=None,
                                                             min_child_weight=None,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=None,
                                                             n_jobs=-1,
                                                             num_parallel_tree=None,
                                                             random_state=100, ...)),
                                               ('Random Forest Regressor',
                                                RandomForestRegressor(n_jobs=-1,
                                                                      random_state=100))],
                                   final_estimator=LinearRegression(n_jobs=-1),
                                   n_jobs=1, passthrough=True))])

In [ ]:
tmp_test

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
0,I,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552
1,I,1.1625,0.8875,0.2750,15.507176,7.030676,3.246018,3.968930
2,F,1.2875,0.9875,0.3250,14.571643,5.556502,3.883882,4.819415
3,F,1.5500,0.9875,0.3875,28.377849,13.380964,6.548735,7.030676
4,I,1.1125,0.8500,0.2625,11.765042,5.528153,2.466407,3.331066
...,...,...,...,...,...,...,...,...
49363,F,1.3000,1.0375,0.3250,16.315137,6.690482,5.173784,3.756309
49364,I,1.0375,0.7625,0.2625,10.276694,4.436697,1.998640,3.543687
49365,F,1.4875,1.1625,0.3625,31.382897,11.396499,6.846404,8.788345
49366,F,1.2375,0.9500,0.2875,15.663099,6.095142,3.727959,4.961163


In [ ]:
tmp_test.isin([np.inf, -np.inf]).sum()

In [ ]:
pred_test = predict_model(stack_model, data = tmp_test)
pred_test

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,prediction_label
0,I,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552,7.617479
1,I,1.1625,0.8875,0.2750,15.507176,7.030676,3.246018,3.968930,7.797796
2,F,1.2875,0.9875,0.3250,14.571643,5.556502,3.883882,4.819415,10.813618
3,F,1.5500,0.9875,0.3875,28.377850,13.380964,6.548735,7.030676,9.687049
4,I,1.1125,0.8500,0.2625,11.765042,5.528152,2.466407,3.331066,7.504417
...,...,...,...,...,...,...,...,...,...
49363,F,1.3000,1.0375,0.3250,16.315138,6.690482,5.173784,3.756309,9.493702
49364,I,1.0375,0.7625,0.2625,10.276693,4.436697,1.998640,3.543688,8.060650
49365,F,1.4875,1.1625,0.3625,31.382896,11.396499,6.846404,8.788345,13.239380
49366,F,1.2375,0.9500,0.2875,15.663098,6.095142,3.727959,4.961163,9.947388


In [ ]:
prediction_round = round(pred_test.prediction_label)

In [ ]:
label.unique() # 기존 라벨은 모두 정수이다.

array([ 9,  8, 11, 10, 12,  7,  5, 14,  6, 13, 16,  4, 18, 15, 17,  3, 19,
       23, 21, 20, 26, 22, 27, 24, 25, 29,  2,  1])

# Make Submission

In [ ]:
submit = pd.read_csv(join(ROOT_PATH, 'data','sample_submission.csv'))

In [ ]:
submit['Age'] = pred_test.prediction_label

In [ ]:
submit.to_csv(join(ROOT_PATH, 'submissions','submission_1.csv'), index=False) # 1.4031

In [ ]:
submit['Age'] = prediction_round

In [ ]:
submit.to_csv(join(ROOT_PATH, 'submissions','submission_2.csv'), index=False) # 1.38366